In [1]:
from dorna2 import Dorna
from camera import Camera
from dorna_vision import Detection
import tools
import config
import time
import numpy as np
import math

#####################################################################
# Connect to the robot:
robot = Dorna()
robot_connected = False
connection_try_counter = 0

while not robot_connected:
    if robot.connect(config.robot["ip"]):
        print("Robot connected")
        robot_connected = True
    else:
        print("Robot connection failed")

    if not robot_connected:
        connection_try_counter += 1
        time.sleep(1)
        if connection_try_counter == 100:
            print("Maximum connection attempts reached")

# Connect and initialize the camera
camera = Camera()
try:
    camera.connect()
except Exception as e:
    print(f"Camera connection failed: {e}")
else:
    print("Camera connected")

# ROI_corners = [[190, 479], [190, 25], [700, 25], [700, 479]] #Original
ROI_corners = [[226, 18], [624, 18], [624, 470], [226, 470]] #New
# Ind_ROI_corners = [[409, 203], [454, 225], [432, 278], [383, 256]] #Original
Ind_ROI_corners = [[409, 103], [554, 225], [432, 378], [283, 256]] #New

# Detections:
preset_general = {'roi': {'corners': ROI_corners, 'inv': False, 'crop': True, 'offset': 0}, "camera_mount":{"type": "dorna_ta_j4","ej": [0 ,0, 0, 0, 0, 0, 0, 0],"T": [48.05923462271373, 30.4362801279415, -5.585734144443932, 0.4111487682507738, -0.18114590496686764, 90.05010215826098]}, 'display': {'label': -1, 'save_img': True, 'save_img_roi': True}, 'detection': {'cmd': 'od', 'path': 'auto_sampler_vial.pkl', 'conf': 0.8, 'cls': []}}
detection_general = Detection(camera=camera, robot=robot, **preset_general)

preset_individual = {'roi': {'corners': Ind_ROI_corners, 'inv': False, 'crop': True, 'offset': 0}, "camera_mount":{"type": "dorna_ta_j4","ej": [0 ,0, 0, 0, 0, 0, 0, 0],"T": [48.05923462271373, 30.4362801279415, -5.585734144443932, 0.4111487682507738, -0.18114590496686764, 90.05010215826098]}, 'display': {'label': 0, 'save_img': False, 'save_img_roi': True}, 'detection': {'cmd': 'od', 'path': 'auto_sampler_vial.pkl', 'conf': 0.8, 'cls': []}}
detection_individual = Detection(camera=camera, robot=robot, **preset_individual)

#Go to initial position:
# tools.safe_init(config=config, robot=robot)
tools.output(config.robot_gripper["open"], robot)
tools.output(config.decapper["open"], robot)
tools.take_init_picture(config=config, robot=robot)
print("Joints before picture: ", robot.get_all_joint())
print("Target joints: ", config.camera["joints"])

# Call the detection
retval = detection_general.run()
print("Vials detected: ", len(retval))

camera_data = detection_general.get_camera_data()

# Initialize control variables
# Calculate the position of the robot's x offset in the world frame
tools.update_robot_frame(config=config)

#Generate target coordinates:
Vials_coords, Caps_coords = tools.generate_vials_caps_coords() # This function outputs Vial and Caps coords. After vision was introduced, these Vial coords are no longer needed
vials_coords_vision = []

for x in retval:
    # Correct xyz:
    detection_xyz = x["xyz"]
    pxl = x["center"]

    xyz = tools.correct_vision_xyz(config=config, vision_xyz=detection_xyz, pxl=pxl, camera=camera, camera_data=camera_data, detection=detection_general)

    # print("Corrected xyz: ", xyz)

    xyz_world = tools.translate_vis_to_world(config=config, xyz=xyz)
    # print("Translated xyz in world: ", xyz_world)

    vials_coords_vision.append(xyz_world)

print("Vial coords from vision: ")
for x in vials_coords_vision: print(x)

sorted_vials = tools.sort_vials(config=config, points=vials_coords_vision)

vials_to_do = len(sorted_vials)
print("Vials to do: ", vials_to_do)
ind = 0

print("Initialization completed")

for i in range(vials_to_do):
    if i < ind:
        continue
    elif i > ind:
        ind = i

    start_time = time.time()
    # tools.update_vial_frame(config=config, vial_frame=Vials_coords[ind])
    tools.update_camera_frame(config=config, camera_frame=sorted_vials[ind])
    tools.update_cap_frame(config=config, cap_frame=Caps_coords[ind])

    tools.take_ind_picture(config=config, robot=robot)
    retval_ind = detection_general.run()
    print("Individual vials detected: ", len(retval_ind))

    ind_vis_vial_coords = []
    min_dist = 1000
    correct_index = 0
    xyz_to_use = [0 , 0, 0]

    for j in range(len(retval_ind)):
        current_ind_xyz = retval_ind[j]["xyz"]
        # print("Ind detection xyz: ", current_ind_xyz)
        pxl = retval_ind[j]["center"]

        xyz = tools.correct_vision_xyz(config=config, vision_xyz=current_ind_xyz, pxl=pxl, camera=camera, camera_data=camera_data, detection=detection_general)

        # print("Corrected ind xyz: ", xyz)

        xyz_world = tools.translate_vis_to_world(config=config, xyz=xyz)
        ind_vis_vial_coords.append(xyz_world)
        # print("Translated ind xyz in world: ", xyz_world)

        # dist = math.dist(sorted_vials[ind][:2], xyz_world[:2])
        # if dist < min_dist:
        #     min_dist = dist
        #     correct_index = j
        #     xyz_to_use = xyz_world
    
    xyz_to_use = min(ind_vis_vial_coords, key=lambda p: math.dist(sorted_vials[ind][:2], p[:2]))

    print("Points identified: ", ind_vis_vial_coords)

    print("Closest point: ", xyz_to_use, ". Final dist: ", math.dist(sorted_vials[ind][:2], xyz_to_use[:2]))

    # print("Final index: ", correct_index, ". Final dist: ", min_dist, ". xyz to use: ", xyz_to_use)

    tools.update_vial_frame_vision(config=config, vial_frame_vision=xyz_to_use)

    # break

    #Sequence:
    tools.vial_tray_to_decapper(config=config, robot=robot)
    tools.cap_to_vial(config=config, robot=robot)
    tools.capping(config=config, robot=robot)
    tools.vial_to_tray(config=config, robot=robot)
    end_time = time.time()

    print("Vial processing completed")
    print("Pick and place movement execution time: ", end_time-start_time)

print("Sequence completed")
camera.close()

Robot connected
Camera connected
Joints before picture:  [-64.907227, 6.591797, -109.182129, 0.043945, 12.612305, -64.995117, 485.00625, 0.0]
Target joints:  [-64.907227, 6.569824, -109.138184, 0.021973, 12.546387, -64.929199]
Vials detected:  10
Vial coords from vision: 
[780.0218247594561, -301.20125072241626, -52.62132835965461]
[795.2517738151512, -256.51296065687075, -52.50880158856832]
[750.4521025633248, -285.01197552175466, -52.83074535583374]
[809.3963975246289, -286.0449108038164, -52.41054457219521]
[824.5297581581779, -316.922061894203, -52.30535591675415]
[765.9590176566076, -240.91709070259876, -52.71629594522524]
[809.6404463791362, -227.03234593803475, -52.403611593049746]
[809.7835133749279, -271.4105624253674, -52.40649655739121]
[766.436927527103, -211.7867061182337, -52.71032495587964]
[795.3856728569469, -212.72814941592767, -52.50399398470462]
Found 6 rows
Found 8 columns
Vials to do:  10
Initialization completed
Camera in world position updated
Cap in world posit

KeyboardInterrupt: 

In [2]:
points = [[749.789699039907, -313.74740303626584, -52.933397817896136],
[794.0251110906863, -301.04957821401774, -52.61965243977518],
[765.7551895993375, -241.25389672551188, -52.7905988718655],
[794.4940460165855, -255.66302523511678, -52.596628180894285],
[779.3742281861562, -285.12352198284356, -52.71480238433186],
[824.8811415744696, -213.29490688661915, -52.36645782659154],
[810.4641186824309, -227.25102879392182, -52.472986092526426],
[823.7720613541218, -316.0721926603738, -52.41892473013432],
[751.472067765813, -211.42521169851807, -52.877134529899465]]
print("Original points:")

for x in points: print(x)

print("Sorted points:")
sorted_points = sorted(points, key=lambda p: p[0])
for x in sorted_points: print(x)

Original points:
[749.789699039907, -313.74740303626584, -52.933397817896136]
[794.0251110906863, -301.04957821401774, -52.61965243977518]
[765.7551895993375, -241.25389672551188, -52.7905988718655]
[794.4940460165855, -255.66302523511678, -52.596628180894285]
[779.3742281861562, -285.12352198284356, -52.71480238433186]
[824.8811415744696, -213.29490688661915, -52.36645782659154]
[810.4641186824309, -227.25102879392182, -52.472986092526426]
[823.7720613541218, -316.0721926603738, -52.41892473013432]
[751.472067765813, -211.42521169851807, -52.877134529899465]
Sorted points:
[749.789699039907, -313.74740303626584, -52.933397817896136]
[751.472067765813, -211.42521169851807, -52.877134529899465]
[765.7551895993375, -241.25389672551188, -52.7905988718655]
[779.3742281861562, -285.12352198284356, -52.71480238433186]
[794.0251110906863, -301.04957821401774, -52.61965243977518]
[794.4940460165855, -255.66302523511678, -52.596628180894285]
[810.4641186824309, -227.25102879392182, -52.47298609

In [4]:
# Grid spacing
grid_spacing = 12  # Adjust based on your actual grid spacing

def cluster_coordinates(values, spacing, tolerance=0.6):
    """Cluster coordinates into grid positions"""
    sorted_vals = sorted(values)
    clusters = []
    current_cluster = [sorted_vals[0]]
    
    for val in sorted_vals[1:]:
        if val - current_cluster[-1] < spacing * tolerance:
            current_cluster.append(val)
        else:
            clusters.append(current_cluster)
            current_cluster = [val]
    clusters.append(current_cluster)
    
    # Return average of each cluster
    return [sum(c) / len(c) for c in clusters]

# Extract x and y coordinates
x_coords = [p[0] for p in points]  # More positive = bottom, more negative = top
y_coords = [p[1] for p in points]  # More positive = left, more negative = right

# Find grid columns and rows (reference positions)
# Sort x in reverse (top to bottom: most positive last)
# Sort y in reverse (left to right: most positive first)
column_centers = cluster_coordinates(y_coords, grid_spacing)  # Y defines columns (left-right)
row_centers = cluster_coordinates(x_coords, grid_spacing)     # X defines rows (top-bottom)

# Reverse row_centers so index 0 = top (most negative X)
row_centers.reverse()
# Reverse column_centers so index 0 = left (most positive Y)
column_centers.reverse()

print(f"Found {len(row_centers)} rows")
print(f"Found {len(column_centers)} columns")

# Assign each point to nearest grid position
def assign_to_grid(point, row_centers, column_centers):
    x, y, z = point[0], point[1], point[2]
    
    # Find nearest row (based on X) and column (based on Y)
    row_idx = min(range(len(row_centers)), 
                  key=lambda i: abs(row_centers[i] - x))
    col_idx = min(range(len(column_centers)), 
                  key=lambda i: abs(column_centers[i] - y))
    
    return (row_idx, col_idx, point)

# Assign all points to grid
grid_points = [assign_to_grid(p, row_centers, column_centers) for p in points]

# Sort by row (top to bottom), then by column (left to right)
sorted_grid_points = sorted(grid_points, key=lambda x: (x[0], x[1]))

# Extract just the points in the correct order
sorted_points = [p[2] for p in sorted_grid_points]

print("\nSorted points (left to right, top to bottom):")
for i, point in enumerate(sorted_points, 1):
    print(f"{i}. {point}")

Found 6 rows
Found 7 columns

Sorted points (left to right, top to bottom):
1. [824.8811415744696, -213.29490688661915, -52.36645782659154]
2. [823.7720613541218, -316.0721926603738, -52.41892473013432]
3. [810.4641186824309, -227.25102879392182, -52.472986092526426]
4. [794.4940460165855, -255.66302523511678, -52.596628180894285]
5. [794.0251110906863, -301.04957821401774, -52.61965243977518]
6. [779.3742281861562, -285.12352198284356, -52.71480238433186]
7. [765.7551895993375, -241.25389672551188, -52.7905988718655]
8. [751.472067765813, -211.42521169851807, -52.877134529899465]
9. [749.789699039907, -313.74740303626584, -52.933397817896136]


[[824.8811415744696, -213.29490688661915, -52.36645782659154], [823.7720613541218, -316.0721926603738, -52.41892473013432], [810.4641186824309, -227.25102879392182, -52.472986092526426], [794.4940460165855, -255.66302523511678, -52.596628180894285], [794.0251110906863, -301.04957821401774, -52.61965243977518], [779.3742281861562, -285.12352198284356, -52.71480238433186], [765.7551895993375, -241.25389672551188, -52.7905988718655], [751.472067765813, -211.42521169851807, -52.877134529899465], [749.789699039907, -313.74740303626584, -52.933397817896136]]


In [1]:
from dorna2 import Dorna
from camera import Camera
from dorna_vision import Detection
import tools
import config
import time

#####################################################################
# Connect to the robot:
robot = Dorna()
robot_connected = False
connection_try_counter = 0

while not robot_connected:
    if robot.connect(config.robot["ip"]):
        print("Robot connected")
        robot_connected = True
    else:
        print("Robot connection failed")

    if not robot_connected:
        connection_try_counter += 1
        time.sleep(1)
        if connection_try_counter == 100:
            print("Maximum connection attempts reached")

# Connect and initialize the camera
camera = Camera()
try:
    camera.connect()
except Exception as e:
    print(f"Camera connection failed: {e}")
else:
    print("Camera connected")

# Detection 
preset = {'roi': {'corners': [[240, 479], [240, 55], [720, 55], [720, 479]], 'inv': False, 'crop': True, 'offset': 0}, "camera_mount":{"type": "dorna_ta_j4","ej": [0 ,0, 0, 0, 0, 0, 0, 0],"T": [48.05923462271373, 30.4362801279415, -5.585734144443932, 0.4111487682507738, -0.18114590496686764, 90.05010215826098]}, 'display': {'label': 0, 'save_img': True, 'save_img_roi': True}, 'detection': {'cmd': 'od', 'path': 'auto_sampler_vial.pkl', 'conf': 0.8, 'cls': []}}
detection = Detection(camera=camera, robot=robot, **preset)

#Go to initial position:
# tools.safe_init(config=config, robot=robot)
tools.take_init_picture(config=config, robot=robot)

# Call the detection
retval = detection.run()
for x in retval: print(x)

Robot connected
Camera connected
{'timestamp': 1760147256.6294243, 'cls': 'vial', 'conf': 0.86517333984375, 'center': [485, 304], 'corners': [[472, 292], [497, 292], [497, 317], [472, 317]], 'color': (0, 252, 199), 'id': 0, 'ej': [0, 0, 0, 0, 0, 0, 0, 0], 'xyz': [164.11721929819666, -250.94807363943616, 88.88411329371549]}


In [ ]:
import numpy as np
import math

detection_xyz = retval[0]["xyz"]
print("Detection xyz: ", detection_xyz)
pxl = retval[0]["center"]
camera_data = detection.get_camera_data()
z_guess = 141.5 #Ensures match on y
z_gt = (z_guess, z_guess)
xyz_target_to_cam = camera.xyz(pxl=pxl, depth_frame=camera_data["depth_frame"], depth_int=camera_data["depth_int"], z_gt=z_gt)

T_target_to_cam = detection.kinematic.xyzabc_to_mat(np.concatenate((np.array(xyz_target_to_cam[0]), np.array([0, 0, 0]))))

# apply frame
T_target_to_frame = np.matmul(detection.frame_mat_inv, T_target_to_cam)
xyz_target_to_frame = detection.kinematic.mat_to_xyzabc(T_target_to_frame).tolist()
xyz = xyz_target_to_frame[0:3]
print("Corrected xyz: ", xyz)

tools.update_robot_frame(config=config)
req_x_off = config.robot["base_in_world"][0] + config.camera["aux"][0]

real_xy=[31*25+12.5-(1.4+12.5)/2-req_x_off, -(10*25+12.5)+(1.4+12.5)/2]
print("real xy: ", real_xy)

print("Distance to detection xy: ", math.dist(detection_xyz[:2], real_xy))
print("Distance to corrected xy: ", math.dist(xyz[:2], real_xy))

Detection xyz:  [164.11721929819666, -250.94807363943616, 88.88411329371549]
Corrected xyz:  [138.35536770193252, -256.0836568415051, -52.793436490349094]
real xy:  [137.82499999999993, -255.55]
Distance to detection xy:  26.69191866188193
Distance to corrected xy:  0.7523825647491094


In [3]:
# Initialize control variables
# Calculate the position of the robot's x offset in the world frame
tools.update_robot_frame(config=config)

#Generate target coordinates:
Vials_coords, Caps_coords = tools.generate_vials_caps_coords()
Vials_coords_vision = []

Vials_coords_vision.append(xyz)

# for x in retval:
#     Vials_coords_vision.append(x["xyz"][:2])

print("Vial coords from vision: ", Vials_coords_vision)

# vials_to_do = 9
vials_to_do = len(Vials_coords_vision)
ind = 0

# camera.close()
print("Initialization completed")

for i in range(vials_to_do):
    if i < ind:
        continue
    elif i > ind:
        ind = i

    start_time = time.time()
    # tools.update_vial_frame(config=config, vial_frame=Vials_coords[ind])
    tools.update_vial_frame_vision(config=config, vial_frame_vision=Vials_coords_vision[ind])
    tools.update_cap_frame(config=config, cap_frame=Caps_coords[ind])

    #Sequence:
    tools.vial_tray_to_decapper(config=config, robot=robot)
    tools.cap_to_vial(config=config, robot=robot)
    tools.capping(config=config, robot=robot)
    tools.vial_to_tray(config=config, robot=robot)
    end_time = time.time()

    print("Vial processing completed")
    print("Pick and place movement execution time: ", end_time-start_time)

print("Sequence completed")
camera.close()

Vial coords from vision:  [[138.35536770193252, -256.0836568415051, -52.793436490349094]]
Initialization completed
Vial frame:  [781.0803677019326, -256.0836568415051, 0, 180, 0, 0] Vial aux:  [485, 0]
Vial in world position updated
Cap in world position updated
Vial processing completed
Pick and place movement execution time:  47.94451951980591
Sequence completed


True